In [1]:
pip install scipy haversine numpy pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

# data = pd.read_csv('/Users/hayoun/Downloads/fulldata_template0918.csv')

### 카페 개수, 쓰레기통 개수 세기 - datanew0921로 저장해둠

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21325 entries, 0 to 21324
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cafe_nm                      21325 non-null  object 
 1   address                      21325 non-null  object 
 2   Longitude                    21325 non-null  float64
 3   Latitude                     21325 non-null  float64
 4   법정동코드                        21325 non-null  int64  
 5   본번                           21325 non-null  int64  
 6   부번                           17234 non-null  float64
 7   Unnamed: 7                   0 non-null      float64
 8   가까운 지하철역(필요X)                0 non-null      float64
 9   지하철역 거리                      0 non-null      float64
 10  가까운 버스정류장(필요X)               0 non-null      float64
 11  버스정류장 거리                     0 non-null      float64
 12  매장크기                         0 non-null      float64
 13  동이름(필요X)        

In [3]:
datanew = data.iloc[:,[0,1,2,3,4]]
datanew.head()

,cafe_nm,address,Longitude,Latitude,법정동코드
0,카페숲,서울특별시 금천구 시흥대로40길,126.909939,37.446689,1154510300
1,매머드커피외대,서울특별시 동대문구 천장산로7길,127.056962,37.598493,1123011000
2,포르마레,서울특별시 강동구 성내로9길,127.125070,37.530474,1174010800
3,카페브릿지,서울특별시 영등포구 여의대로,126.929094,37.525184,1156011000
4,더웨이유통할리스커피,서울특별시 중구 다산로,127.010407,37.553952,1114016200


In [4]:
from scipy.spatial import KDTree
from haversine import haversine, Unit

cafe_coords = np.array(datanew[['Latitude', 'Longitude']])

### 기본 코드

In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [7]:
from tqdm import tqdm

In [8]:
# 데이터 청크로 나누기
num_chunks = 6
chunks = np.array_split(datanew, num_chunks)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [11]:
import os

In [12]:
# 저장 경로 설정
# save_path = '/Users/hayoun/Desktop/DAB/data/'

# 각 청크를 처리한 후, 해당 청크의 결과를 지정된 경로에 CSV 파일로 저장
def process_and_save_chunk(chunk, chunk_id, data, radius=0.3):
    result = []
    # tqdm을 사용하여 진행률 표시
    for idx, row in tqdm(chunk.iterrows(), total=chunk.shape[0], desc=f"Processing chunk {chunk_id}"):
        count = 0
        cafe_location = (row['Latitude'], row['Longitude'])
        for other_idx, other_row in data.iterrows():
            if idx != other_idx:
                other_location = (other_row['Latitude'], other_row['Longitude'])
                distance = haversine(cafe_location, other_location)
                if distance <= radius:
                    count += 1
        result.append(count)

    # 처리된 결과를 데이터프레임으로 변환하여 저장
    chunk['cafeinrange'] = result
    chunk_filename = os.path.join(save_path, f'chunk_{chunk_id}_result.csv')
    chunk.to_csv(chunk_filename, index=False)
    print(f"Chunk {chunk_id} saved to {chunk_filename}")
    return chunk_filename


In [13]:
# 4번째 청크부터 처리 (index 3부터 시작)
saved_files = []
for i, chunk in enumerate(chunks[3:], start=4):  # 4번째 청크부터 시작
    chunk_filename = process_and_save_chunk(chunk, i, datanew)
    saved_files.append(chunk_filename)

# 파일이 저장된 경로 확인
print(f"All chunks have been processed and saved to {save_path}.")

Processing chunk 4: 100%|██████████| 3554/3554 [16:38<00:00,  3.56it/s]


Chunk 4 saved to /Users/hayoun/Desktop/DAB/data/chunk_4_result.csv


Processing chunk 5: 100%|██████████| 3554/3554 [16:43<00:00,  3.54it/s]


Chunk 5 saved to /Users/hayoun/Desktop/DAB/data/chunk_5_result.csv


Processing chunk 6: 100%|██████████| 3554/3554 [16:41<00:00,  3.55it/s]

Chunk 6 saved to /Users/hayoun/Desktop/DAB/data/chunk_6_result.csv
All chunks have been processed and saved to /Users/hayoun/Desktop/DAB/data/.


#### 쓰레기통 개수

In [18]:
# 300m 반경 내 쓰레기통 개수를 계산하는 함수
def count_trashbins_near_cafes(cafe_coords, trashbin_coords, radius=300):
    """
    각 카페 근방 300m 내에 있는 쓰레기통의 개수를 계산합니다.
    
    Parameters:
    cafe_coords (DataFrame): 카페 좌표 데이터프레임
    trashbin_coords (DataFrame): 쓰레기통 좌표 데이터프레임
    radius (float): 탐색 반경 (단위: m), 기본값 300m
    
    Returns:
    list: 각 카페 근방의 쓰레기통 개수
    """
    trashbin_count = []
    
    # 각 카페마다 300m 반경 내의 쓰레기통 개수를 계산
    for idx, cafe_row in tqdm(cafe_coords.iterrows(), total=cafe_coords.shape[0], desc="Processing cafes"):
        cafe_location = (cafe_row['Latitude'], cafe_row['Longitude'])
        count = 0
        
        # 모든 쓰레기통과의 거리를 일일이 계산
        for _, trash_row in trashbin_coords.iterrows():
            trash_location = (trash_row['Latitude'], trash_row['Longitude'])
            distance = haversine(cafe_location, trash_location, unit=Unit.METERS)
            
            # 300m 이내의 쓰레기통만 카운트
            if distance <= radius:
                count += 1
        
        trashbin_count.append(count)
    
    return trashbin_count


In [19]:
# 300m 반경 내 쓰레기통 개수 계산
trashbin_count = count_trashbins_near_cafes(datanew[['Latitude', 'Longitude']], trashbin[['Latitude', 'Longitude']], radius=300)

# 결과를 데이터프레임에 추가
datanew['trashbins_in_range'] = trashbin_count

# 최종 결과 출력
print(datanew[['cafe_nm', 'trashbins_in_range']])

Processing cafes: 100%|██████████| 21325/21325 [19:46<00:00, 17.97it/s]

          cafe_nm  trashbins_in_range
0             카페숲                   0
1         매머드커피외대                   1
2            포르마레                   0
3           카페브릿지                   2
4      더웨이유통할리스커피                   7
...           ...                 ...
21320   빽다방빨래골입구점                   1
21321         청23                   5
21322        피트커피                   0
21323    빽다방동여의도점                   4
21324       보광동커피                   2

[21325 rows x 2 columns]



/var/folders/xx/z7zmyqq92gs28yp3ngzz0z4h0000gn/T/ipykernel_2652/2009477366.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['trashbins_in_range'] = trashbin_count


#### 합치기

In [20]:
# 모든 청크 파일을 병합하기 위한 리스트
chunk_files = [f'chunk_{i}_result.csv' for i in range(1, 7)]  # 6개의 청크 파일
chunk_dfs = []

# 각 청크 파일을 불러와 병합
for chunk_file in chunk_files:
    chunk_path = os.path.join(save_path, chunk_file)
    
    if os.path.exists(chunk_path):
        chunk_df = pd.read_csv(chunk_path)
        chunk_dfs.append(chunk_df)
    else:
        print(f"File {chunk_file} not found!")

# 청크들을 하나로 병합
merged_chunks = pd.concat(chunk_dfs, ignore_index=True)

In [22]:
merged_chunks.head()

,cafe_nm,address,Longitude,Latitude,법정동코드,cafeinrange
0,카페숲,서울특별시 금천구 시흥대로40길,126.909939,37.446689,1154510300,2
1,매머드커피외대,서울특별시 동대문구 천장산로7길,127.056962,37.598493,1123011000,23
2,포르마레,서울특별시 강동구 성내로9길,127.125070,37.530474,1174010800,26
3,카페브릿지,서울특별시 영등포구 여의대로,126.929094,37.525184,1156011000,33
4,더웨이유통할리스커피,서울특별시 중구 다산로,127.010407,37.553952,1114016200,34


In [21]:
datanew.head()

,cafe_nm,address,Longitude,Latitude,법정동코드,trashbins_in_range
0,카페숲,서울특별시 금천구 시흥대로40길,126.909939,37.446689,1154510300,0
1,매머드커피외대,서울특별시 동대문구 천장산로7길,127.056962,37.598493,1123011000,1
2,포르마레,서울특별시 강동구 성내로9길,127.125070,37.530474,1174010800,0
3,카페브릿지,서울특별시 영등포구 여의대로,126.929094,37.525184,1156011000,2
4,더웨이유통할리스커피,서울특별시 중구 다산로,127.010407,37.553952,1114016200,7


In [23]:
# datanew에 cafeinrange 컬럼을 붙이기 위해 기존 datanew와 병합
# 두 데이터프레임은 인덱스 또는 고유 ID를 기준으로 병합해야 함
# 여기서는 단순히 인덱스 기준으로 병합
datanew['cafeinrange'] = merged_chunks['cafeinrange'].values

# 결과 확인
print(datanew)

          cafe_nm            address   Longitude   Latitude       법정동코드  \
0             카페숲  서울특별시 금천구 시흥대로40길  126.909939  37.446689  1154510300   
1         매머드커피외대  서울특별시 동대문구 천장산로7길  127.056962  37.598493  1123011000   
2            포르마레    서울특별시 강동구 성내로9길  127.125070  37.530474  1174010800   
3           카페브릿지    서울특별시 영등포구 여의대로  126.929094  37.525184  1156011000   
4      더웨이유통할리스커피       서울특별시 중구 다산로  127.010407  37.553952  1114016200   
...           ...                ...         ...        ...         ...   
21320   빽다방빨래골입구점      서울특별시 강북구 삼양로  127.017708  37.629745  1130510300   
21321         청23      서울특별시 마포구 연희로  126.926523  37.560923  1144012100   
21322        피트커피  서울특별시 강동구 고덕비즈밸리로  127.160262  37.565439  1174010200   
21323    빽다방동여의도점   서울특별시 영등포구 국제금융로  126.931778  37.520198  1156011000   
21324       보광동커피      서울특별시 용산구 장문로  126.999566  37.525973  1117013600   

       trashbins_in_range  cafeinrange  
0                       0            2  
1                

/var/folders/xx/z7zmyqq92gs28yp3ngzz0z4h0000gn/T/ipykernel_2652/2598292986.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['cafeinrange'] = merged_chunks['cafeinrange'].values


In [24]:
datanew.to_csv('/Users/hayoun/Desktop/DAB/data/datanew0921.csv')

## 공공시설 개수 세기

In [3]:
# 데이터 불러오기
# fulldata = pd.read_csv('/Users/hayoun/Desktop/DAB/data/datanew0921.csv')

# public = pd.read_csv('/Users/hayoun/Desktop/DAB/data/서울시 시설물 정보.csv', encoding ='euc-kr' )

In [15]:
fulldata.head()

,Unnamed: 0,cafe_nm,address,Longitude,Latitude,법정동코드,trashbins_in_range,cafeinrange
0,0,카페숲,서울특별시 금천구 시흥대로40길,126.909939,37.446689,1154510300,0,2
1,1,매머드커피외대,서울특별시 동대문구 천장산로7길,127.056962,37.598493,1123011000,1,23
2,2,포르마레,서울특별시 강동구 성내로9길,127.125070,37.530474,1174010800,0,26
3,3,카페브릿지,서울특별시 영등포구 여의대로,126.929094,37.525184,1156011000,2,33
4,4,더웨이유통할리스커피,서울특별시 중구 다산로,127.010407,37.553952,1114016200,7,34


In [4]:
public.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3606 entries, 0 to 3605
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   아이디        3606 non-null   int64  
 1   새주소 아이디    0 non-null      float64
 2   시설 아이디     3606 non-null   object 
 3   시설명        3606 non-null   object 
 4   위도         3606 non-null   float64
 5   경도         3606 non-null   float64
 6   건물군여부      2260 non-null   object 
 7   시설용도분류     3606 non-null   object 
 8   소재지 도로명주소  3494 non-null   object 
 9   소재지 지번주소   3606 non-null   object 
 10  기타         89 non-null     object 
 11  국가지점번호     3606 non-null   object 
 12  데이터 기준일자   3606 non-null   object 
dtypes: float64(3), int64(1), object(9)
memory usage: 366.4+ KB


In [8]:
public.head()

,아이디,새주소 아이디,시설 아이디,시설명,위도,경도,건물군여부,시설용도분류,소재지 도로명주소,소재지 지번주소,기타,국가지점번호,데이터 기준일자
0,3007,NaN,P0877,꿈동산어린이공원,37.650288,127.036860,BG_AA,FU_BI,서울특별시 도봉구 도봉로110나길 99,서울특별시 도봉구 창동 663-1,NaN,다사 5914 6130,2022-12-07 00:00:00.0
1,3008,NaN,P0878,미화어린이공원,37.654945,127.039690,BG_AA,FU_BI,서울특별시 도봉구 도봉로 564,서울특별시 도봉구 창동 715-17,NaN,다사 5939 6181,2022-12-07 00:00:00.0
2,3009,NaN,P0879,꿈동산어린이공원,37.573290,127.045840,BG_AA,FU_BI,서울특별시 동대문구 황물로 42,서울특별시 동대문구 답십리동 1002-1,NaN,다사 5989 5275,2022-12-07 00:00:00.0
3,3010,NaN,P0880,새움어린이공원,37.570760,127.048676,BG_AA,FU_BI,서울특별시 동대문구 천호대로55길 11,서울특별시 동대문구 답십리동 1009-1,NaN,다사 6014 5247,2022-12-07 00:00:00.0
4,3012,NaN,P0882,동산어린이공원,37.566967,127.057540,BG_AA,FU_BI,서울특별시 동대문구 전농로2나길 7,서울특별시 동대문구 답십리동 998-1,NaN,다사 6092 5204,2022-12-07 00:00:00.0


In [9]:
public['시설용도분류'].unique()
# FU_BA가 '대규모점포'

array(['FU_BI', 'FU_BA', 'FU_BC', 'FU_BB', 'FU_BD', 'FU_BE', 'FU_BG',
       'FU_BF', 'FU_BH', 'FU_BJ'], dtype=object)

In [10]:
public = public[public['시설용도분류'] != 'FU_BA']
public['시설용도분류'].unique()

array(['FU_BI', 'FU_BC', 'FU_BB', 'FU_BD', 'FU_BE', 'FU_BG', 'FU_BF',
       'FU_BH', 'FU_BJ'], dtype=object)

In [11]:
# 시설명, 위도, 경도, 소재지 도로명 주소 열만 남기기
columns_to_keep = ['시설 아이디', '시설명', '위도', '경도', '소재지 도로명주소']
public = public[columns_to_keep]
public.head()

,시설 아이디,시설명,위도,경도,소재지 도로명주소
0,P0877,꿈동산어린이공원,37.650288,127.036860,서울특별시 도봉구 도봉로110나길 99
1,P0878,미화어린이공원,37.654945,127.039690,서울특별시 도봉구 도봉로 564
2,P0879,꿈동산어린이공원,37.573290,127.045840,서울특별시 동대문구 황물로 42
3,P0880,새움어린이공원,37.570760,127.048676,서울특별시 동대문구 천호대로55길 11
4,P0882,동산어린이공원,37.566967,127.057540,서울특별시 동대문구 전농로2나길 7


In [13]:
from haversine import haversine, Unit
from tqdm import tqdm

def count_facilities_near_cafes(cafe_coords, facility_coords, radius=300):
    """
    각 카페 근방 300m 내에 있는 공공시설의 개수를 계산합니다.
    
    Parameters:
    cafe_coords (DataFrame): 카페 좌표 데이터프레임
    facility_coords (DataFrame): 공공시설 좌표 데이터프레임
    radius (float): 탐색 반경 (단위: m), 기본값 300m
    
    Returns:
    list: 각 카페 근방의 공공시설 개수
    """
    facility_count = []
    
    # 각 카페마다 300m 반경 내의 공공시설 개수 계산
    for idx, cafe_row in tqdm(cafe_coords.iterrows(), total=cafe_coords.shape[0], desc="Processing cafes"):
        cafe_location = (cafe_row['Latitude'], cafe_row['Longitude'])
        count = 0
        
        # 모든 공공시설과의 거리를 일일이 계산
        for _, facility_row in facility_coords.iterrows():
            facility_location = (facility_row['위도'], facility_row['경도'])
            distance = haversine(cafe_location, facility_location, unit=Unit.METERS)
            
            # 300m 이내의 공공시설만 카운트
            if distance <= radius:
                count += 1
        
        facility_count.append(count)
    
    return facility_count


In [17]:
# 300m 반경 내 공공시설 개수 계산
facility_count = count_facilities_near_cafes(fulldata[['Latitude', 'Longitude']], public[['위도', '경도']], radius=300)

# 결과를 데이터프레임에 추가
fulldata['facilities_in_range'] = facility_count

# 최종 결과 출력
fulldata.head()

Processing cafes: 100%|██████████| 21325/21325 [10:21<00:00, 34.29it/s]


,Unnamed: 0,cafe_nm,address,Longitude,Latitude,법정동코드,trashbins_in_range,cafeinrange,facilities_in_range
0,0,카페숲,서울특별시 금천구 시흥대로40길,126.909939,37.446689,1154510300,0,2,1
1,1,매머드커피외대,서울특별시 동대문구 천장산로7길,127.056962,37.598493,1123011000,1,23,0
2,2,포르마레,서울특별시 강동구 성내로9길,127.125070,37.530474,1174010800,0,26,4
3,3,카페브릿지,서울특별시 영등포구 여의대로,126.929094,37.525184,1156011000,2,33,0
4,4,더웨이유통할리스커피,서울특별시 중구 다산로,127.010407,37.553952,1114016200,7,34,4


In [18]:
# fulldata.to_csv('/Users/hayoun/Desktop/DAB/data/datanew0922.csv')

### (사용X) KD TREE 활용

In [7]:
def count_cafes_within_radius(cafe_coords, radius=300):
    """
    각 카페 근방 300m 내에 있는 다른 카페의 개수를 구합니다.
    
    Parameters:
    cafe_coords (numpy array): shape (n_cafes, 2), 카페 좌표 데이터
    radius (float): 탐색 반경 (단위: m), 기본값 300m
    
    Returns:
    numpy array: 각 카페 근방의 다른 카페 개수 (n_cafes,)
    """
    # 카페 좌표에 대한 KDTree 생성
    cafe_tree = KDTree(cafe_coords)
    
    # 각 카페에 대해 반경 내 다른 카페의 개수를 저장할 리스트
    cafes_count = []
    
    # 각 카페 좌표에 대해 300m 반경 내의 다른 카페 개수를 계산
    for cafe in cafe_coords:
        # KDTree에서 근사 거리로 후보 찾기 (약 300m 반경으로 검색)
        candidate_indices = cafe_tree.query_ball_point(cafe, r=0.05)  # 0.003도 근사치로 검색
    
        
        # 후보 중 haversine 거리를 다시 계산하여 300m 내 카페 필터링
        count = 0
        for idx in candidate_indices:
            other_cafe = cafe_coords[idx]
            distance = haversine(cafe, other_cafe, unit=Unit.METERS)  # haversine으로 정확한 거리 계산
            if 0 < distance <= radius:  # 자기 자신은 제외하고 300m 내 카페만 카운트
                count += 1
        
        cafes_count.append(count)
    
    return np.array(cafes_count)

# 300m 내 카페 개수 계산
cafes_count = count_cafes_within_radius(cafe_coords, radius=300)

# 결과를 데이터프레임에 추가
datanew['cafes_in_range'] = cafes_count

# 결과 출력
print(datanew[['cafe_nm', 'cafes_in_range']])

          cafe_nm  cafes_in_range
0             카페숲               2
1         매머드커피외대              23
2            포르마레              26
3           카페브릿지              19
4      더웨이유통할리스커피              34
...           ...             ...
21320   빽다방빨래골입구점              10
21321         청23             136
21322        피트커피               4
21323    빽다방동여의도점              59
21324       보광동커피              14

[21325 rows x 2 columns]


/var/folders/xx/z7zmyqq92gs28yp3ngzz0z4h0000gn/T/ipykernel_33599/2593881520.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['cafes_in_range'] = cafes_count


In [8]:
colabdata = pd.read_csv('/Users/hayoun/Downloads/chunk_1_result.csv')
colabdata.head()

,cafe_nm,address,Longitude,Latitude,법정동코드,cafeinrange
0,카페숲,서울특별시 금천구 시흥대로40길,126.909939,37.446689,1154510300,2
1,매머드커피외대,서울특별시 동대문구 천장산로7길,127.056962,37.598493,1123011000,23
2,포르마레,서울특별시 강동구 성내로9길,127.125070,37.530474,1174010800,26
3,카페브릿지,서울특별시 영등포구 여의대로,126.929094,37.525184,1156011000,33
4,더웨이유통할리스커피,서울특별시 중구 다산로,127.010407,37.553952,1114016200,34


In [15]:
trashbin = pd.read_csv('/Users/hayoun/Downloads/trashbin.csv')

trashbin = trashbin.iloc[:,[0,1,2,3,4]]
trashbin.head()

,연번,자치구명,설치위치(도로명 주소),Latitude,Longitude
0,1,종로구,사직로 125,37.576160,126.972866
1,2,종로구,사직로 125,37.576160,126.972866
2,3,종로구,자하문로 28,37.578560,126.971968
3,4,종로구,자하문로 28,37.578560,126.971968
4,5,종로구,자하문로 44,37.579972,126.971553


In [17]:
trashbin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5328 entries, 0 to 5379
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            5328 non-null   int64  
 1   자치구명          5328 non-null   object 
 2   설치위치(도로명 주소)  5328 non-null   object 
 3   Latitude      5328 non-null   float64
 4   Longitude     5328 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 249.8+ KB


In [16]:
trashbin = trashbin.dropna(subset=['Latitude', 'Longitude'])

trashbin_coords = np.array(trashbin[['Latitude', 'Longitude']])

In [11]:
trashbin_coords

array([[ 37.57616  , 126.9728657],
       [ 37.57616  , 126.9728657],
       [ 37.5785599, 126.9719683],
       ...,
       [ 37.5550645, 127.1516047],
       [ 37.5463255, 127.1714618],
       [ 37.5525648, 127.172524 ]])

In [13]:
def count_trashbins_near_cafes(cafe_coords, trashbin_coords, radius=300):
    """
    각 카페 근방 300m 내에 있는 쓰레기통의 개수를 구합니다.
    
    Parameters:
    cafe_coords (numpy array): shape (n_cafes, 2), 카페 좌표 데이터
    trashbin_coords (numpy array): shape (n_trashbins, 2), 쓰레기통 좌표 데이터
    radius (float): 탐색 반경 (단위: m), 기본값 300m
    
    Returns:
    numpy array: 각 카페 근방의 쓰레기통 개수 (n_cafes,)
    """
    # 쓰레기통 좌표에 대한 KDTree 생성
    trashbin_tree = KDTree(trashbin_coords)
    
    # 각 카페에 대해 반경 내 쓰레기통 개수를 저장할 리스트
    trashbin_count = []
    
    # 각 카페 좌표에 대해 300m 반경 내의 쓰레기통 개수를 계산
    for cafe in cafe_coords:
        # KDTree에서 근사 거리로 후보 찾기 (약 300m 반경으로 검색)
        candidate_indices = trashbin_tree.query_ball_point(cafe, r=0.05)  # 0.003도 근사치로 검색
        
        # 후보 중 haversine 거리를 다시 계산하여 300m 내 쓰레기통 필터링
        count = 0
        for idx in candidate_indices:
            trashbin = trashbin_coords[idx]
            distance = haversine(cafe, trashbin, unit=Unit.METERS)  # haversine으로 정확한 거리 계산
            if distance <= radius:  # 300m 내 쓰레기통만 카운트
                count += 1
        
        trashbin_count.append(count)
    
    return np.array(trashbin_count)

# 300m 내 쓰레기통 개수 계산
trashbin_count = count_trashbins_near_cafes(cafe_coords, trashbin_coords, radius=300)

# 결과를 데이터프레임에 추가
datanew['trashbins_in_range'] = trashbin_count

# 결과 출력
print(datanew[['cafe_nm', 'trashbins_in_range']])

          cafe_nm  trashbins_in_range
0             카페숲                   0
1         매머드커피외대                   1
2            포르마레                   0
3           카페브릿지                   2
4      더웨이유통할리스커피                   7
...           ...                 ...
21320   빽다방빨래골입구점                   1
21321         청23                   5
21322        피트커피                   0
21323    빽다방동여의도점                   4
21324       보광동커피                   2

[21325 rows x 2 columns]


/var/folders/xx/z7zmyqq92gs28yp3ngzz0z4h0000gn/T/ipykernel_33599/600496322.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datanew['trashbins_in_range'] = trashbin_count


In [14]:
datanew.to_csv('/Users/hayoun/Desktop/DAB/data/datanew.csv')